In [3]:
#sudo pip install --upgrade google-api-python-client
#sudo pip install oauth2client
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# create an project and get an API key at https://console.developers.google.com
# for youtube data
DEVELOPER_KEY = "XXX"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)
# q: search word
# max_results: maximum videos to scraped, default is 50, as youtube doesn't allow to scrape more than 50
# order: how the videos should be ordered before scraping
# token: if existing, otherwise None
# location, and location_radius are additional filters

#,youtube_api_name = YOUTUBE_API_SERVICE_NAME, youtube_api_version= YOUTUBE_API_VERSION,dev_key=DEVELOPER_KEY
def youtube_search(q, max_results=50, order="relevance",token=None, location=None, location_radius=None,
                   youtube_api_name = None, youtube_api_version= None, dev_key= None, regionCode= None):
    '''
    this function is just about extracting useful info and getting it working
    '''
    # Build the service object
    youtube = build(youtube_api_name, youtube_api_version, developerKey = dev_key)

    # search on the service object for the search word q
    # returning object is a dictionary, which contains information such as 'nextPageToken', 'regionCode', 'pageInfo',
    search_response = youtube.search().list(
    q=q,
    type="video",
    pageToken=token,
    order = order,
    part="id,snippet", # Part signifies the different types of data you want 
    maxResults=max_results,
    location=location,
    locationRadius=location_radius,
    regionCode=regionCode).execute()

    # initalise lists variable to get more organised information
    title = []
    channelId = []
    channelTitle = []
    categoryId = []
    videoId = []
    viewCount = []
    likeCount = []
    dislikeCount = []
    commentCount = []
    favoriteCount = []
    category = []
    tags = []
    videos = []
    description = []
    
    # in each loop search_results contains information of one video
    for search_result in search_response.get("items", []):
        # only look at youtubevideos
        if search_result["id"]["kind"] == "youtube#video":

            # filter the title
            title.append(search_result['snippet']['title']) 

            # filter the videoID
            videoId.append(search_result['id']['videoId'])

            # get dictionary with information on video level
            response = youtube.videos().list(
                part='statistics, snippet',
                id=search_result['id']['videoId']).execute()

            # extract key information of video
            channelId.append(response['items'][0]['snippet']['channelId'])
            channelTitle.append(response['items'][0]['snippet']['channelTitle'])
            categoryId.append(response['items'][0]['snippet']['categoryId'])
            favoriteCount.append(response['items'][0]['statistics']['favoriteCount'])
            viewCount.append(response['items'][0]['statistics']['viewCount'])
            description.append(response['items'][0]['snippet']['description'])
            
            # some statistics are missing, in that case assign value None 
            statisticsVar = response['items'][0]['statistics'].keys()
            if 'likeCount' in statisticsVar:
                likeCount.append(response['items'][0]['statistics']['likeCount'])
            else: 
                likeCount.append(None)
                
            if 'dislikeCount' in statisticsVar:
                dislikeCount.append(response['items'][0]['statistics']['dislikeCount'])
            else: 
                dislikeCount.append(None)
                
            if 'commentCount' in statisticsVar:
                commentCount.append(response['items'][0]['statistics']['commentCount'])
            else: 
                commentCount.append(None)
      
            if 'tags' in response['items'][0]['snippet'].keys():
                tags.append(response['items'][0]['snippet']['tags'])
            else:
                tags.append(None)

    youtube_dict = {'tags':tags,'channelId': channelId,'channelTitle': channelTitle,'categoryId':categoryId,
                    'title':title,'videoId':videoId,'viewCount':viewCount,'likeCount':likeCount,'dislikeCount':dislikeCount,
                    'commentCount':commentCount,'favoriteCount':favoriteCount, 'description':description}

    return youtube_dict

In [5]:
search_global_warming = youtube_search(q = "global warming conspiracy", max_results = 5, youtube_api_name = YOUTUBE_API_SERVICE_NAME, youtube_api_version= YOUTUBE_API_VERSION, dev_key= DEVELOPER_KEY)
df_global_warming = pd.DataFrame(data=search_global_warming)
df_global_warming = df_global_warming[['title','viewCount','channelTitle','commentCount','likeCount','dislikeCount','tags','favoriteCount','videoId','channelId','categoryId','description']]

search_global_warming_USA = youtube_search(q = "global warming conspiracy", max_results = 5, youtube_api_name = YOUTUBE_API_SERVICE_NAME, youtube_api_version= YOUTUBE_API_VERSION, dev_key= DEVELOPER_KEY, regionCode = 'US')
df_global_warming_USA = pd.DataFrame(data=search_global_warming_USA)
df_global_warming_USA = df_global_warming_USA[['title','viewCount','channelTitle','commentCount','likeCount','dislikeCount','tags','favoriteCount','videoId','channelId','categoryId','description']]


In [6]:
display(df_global_warming.head())
display(df_global_warming_USA.head())

,title,viewCount,channelTitle,commentCount,likeCount,dislikeCount,tags,favoriteCount,videoId,channelId,categoryId,description
0,Climate Change a Hoax?,23012,CBS,323,151,66,"[cbsepisode, climate, change, global, warming,...",0,UE6QxBaIEv8,UClzCn8DxRSCuMFv_WfzkcrQ,25,A series of leaked e-mails between climate sci...
1,Climate Change Researcher Describes Challenge ...,61786,The Onion,786,2889,125,"[The Onion, comedy, satire, news, Onion News N...",0,MnX0-TayVjk,UCfAOh2t5DpxVrgS9NQKjC7A,23,Subscribe to The Onion on YouTube: http://bit....
2,Lord Christopher Monckton - Global Warming is ...,138803,ideacity,1384,2632,309,"[ideacity, idea, city, moses znaimer, lord chr...",0,UGqcweY1a3I,UCNdVhGg0-TkkzWf9Gl0djow,27,Lord Christopher Monckton shares his thoughts ...
3,Climate Change and Conspiracy Theories at the ...,607355,Comedy Central,587,7730,187,"[The Jim Jefferies Show, Jim Jefferies, Jim Je...",0,fi8I3reckfQ,UCUsN5ZwHx2kILm84-jPDeXw,23,"As bleaching ravages the Great Barrier Reef, J..."
4,DR GAVIN SCHMIDT - GLOBAL WARMING EXPOSED - Pa...,33312,London Real,356,194,284,"[gavin schmidt, global warming, climate change...",0,j4KjWGr0nBY,UCCZVmatSqIMTTB8uExk8xEg,28,FREE LIVE MASTERCLASS: https://londonreal.tv/m...


,title,viewCount,channelTitle,commentCount,likeCount,dislikeCount,tags,favoriteCount,videoId,channelId,categoryId,description
0,Climate Change Researcher Describes Challenge ...,61786,The Onion,786,2889,125,"[The Onion, comedy, satire, news, Onion News N...",0,MnX0-TayVjk,UCfAOh2t5DpxVrgS9NQKjC7A,23,Subscribe to The Onion on YouTube: http://bit....
1,Climate Change a Hoax?,23012,CBS,323,151,66,"[cbsepisode, climate, change, global, warming,...",0,UE6QxBaIEv8,UClzCn8DxRSCuMFv_WfzkcrQ,25,A series of leaked e-mails between climate sci...
2,Lord Christopher Monckton - Global Warming is ...,138803,ideacity,1384,2632,309,"[ideacity, idea, city, moses znaimer, lord chr...",0,UGqcweY1a3I,UCNdVhGg0-TkkzWf9Gl0djow,27,Lord Christopher Monckton shares his thoughts ...
3,DR GAVIN SCHMIDT - GLOBAL WARMING EXPOSED - Pa...,33312,London Real,356,194,284,"[gavin schmidt, global warming, climate change...",0,j4KjWGr0nBY,UCCZVmatSqIMTTB8uExk8xEg,28,FREE LIVE MASTERCLASS: https://londonreal.tv/m...
4,Climate Change and Conspiracy Theories at the ...,607355,Comedy Central,587,7730,187,"[The Jim Jefferies Show, Jim Jefferies, Jim Je...",0,fi8I3reckfQ,UCUsN5ZwHx2kILm84-jPDeXw,23,"As bleaching ravages the Great Barrier Reef, J..."


In [56]:
print(df_global_warming['description'][df_global_warming['channelTitle']=='CBS'].values)

['A series of leaked e-mails between climate scientists is casting a cloud over the Climate Change Summit. Kimberly Dozier reports whether scientists fudged numbers to over-exaggerate climate change.']


In [57]:
print(df_global_warming_USA['description'][df_global_warming_USA['channelTitle']=='CBS'].values)

['A series of leaked e-mails between climate scientists is casting a cloud over the Climate Change Summit. Kimberly Dozier reports whether scientists fudged numbers to over-exaggerate climate change.']


In [17]:
def comment_threads_list_by_video_id(client, **kwargs):
  # See full sample for function
#     kwargs = remove_empty_kwargs(**kwargs)

    response = client.commentThreads().list(**kwargs).execute()

    return response

comments = comment_threads_list_by_video_id(youtube, part='snippet,replies', videoId='UE6QxBaIEv8')

In [ ]:
def comment_times(comments):
    
    #get comment times and reply times as a proxy for views
    return activity_times

In [37]:
for c, item in enumerate(comments['items']):
    c_time = item['snippet']
    print(c, item['replies']) 
    

0 {'comments': [{'kind': 'youtube#comment', 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/0JuKtduxqE5Oyc9289FrqYm7oBA"', 'id': 'Ugyt0ei9AtZ5iyMqKKF4AaABAg.8pN3O2mP5h28pOQDtwWhxD', 'snippet': {'authorDisplayName': 'Jason Voorhees', 'authorProfileImageUrl': 'https://yt3.ggpht.com/-6e7S7Kx-YTA/AAAAAAAAAAI/AAAAAAAAAAA/vSt8GuA-1gQ/s28-c-k-no-mo-rj-c0xffffff/photo.jpg', 'authorChannelUrl': 'http://www.youtube.com/channel/UChOk7sC-yGyO1JbHCeqDkMQ', 'authorChannelId': {'value': 'UChOk7sC-yGyO1JbHCeqDkMQ'}, 'videoId': 'UE6QxBaIEv8', 'textDisplay': 'I can&#39;t believe there are still some people out there who think &#39;global warming&#39; is real. Like, how can you not see the literal fucking obvious?!', 'textOriginal': "I can't believe there are still some people out there who think 'global warming' is real. Like, how can you not see the literal fucking obvious?!", 'parentId': 'Ugyt0ei9AtZ5iyMqKKF4AaABAg', 'canRate': True, 'viewerRating': 'none', 'likeCount': 0, 'publishedAt': '2018-12-28T06:12:05.00

KeyError: 'replies'

In [43]:
comments['items'][0]['snippet']

{'videoId': 'UE6QxBaIEv8',
 'topLevelComment': {'kind': 'youtube#comment',
  'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/EfFIrMcGqqDeH8j9Dygg0ZavkVM"',
  'id': 'Ugyt0ei9AtZ5iyMqKKF4AaABAg',
  'snippet': {'authorDisplayName': 'Headstash Music',
   'authorProfileImageUrl': 'https://yt3.ggpht.com/--rVceYZwVU8/AAAAAAAAAAI/AAAAAAAAAAA/oZqp69-9g-8/s28-c-k-no-mo-rj-c0xffffff/photo.jpg',
   'authorChannelUrl': 'http://www.youtube.com/channel/UCcaBPC-p5nnDVcwRWo92gOw',
   'authorChannelId': {'value': 'UCcaBPC-p5nnDVcwRWo92gOw'},
   'videoId': 'UE6QxBaIEv8',
   'textDisplay': 'Your thermometer is a hoax. if you are old enough you may have noticed it goes higher than it used to. You may even feel that it is getting warmer. But that is all just an illusion; programming of the matrix, created by scientists who want to make billions, totally unlike fossil fuel companies, who exist only for altruistic purposes.',
   'textOriginal': 'Your thermometer is a hoax. if you are old enough you may have noticed it 